### Evaluating sequencing results for JPEG experimental run

In [26]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [154]:
from utils import get_fastq_records, reverse_complement, create_fasta_file
from clustering import Clustering
from strand_reconstruction import align
import matplotlib.pyplot as plt
import pandas as pd
from checksum import CheckSum4
import uuid
import os
from tqdm import tqdm

In [152]:
filepath = r"C:\Users\Parv\Doc\RA\Projects\sequencingData\JPEGDNA\270325_TWIST_PCR 1\270325_TWIST_PCR\fastq_pass"

In [ ]:
records = []

In [155]:
for file in tqdm(os.listdir(filepath)):
    records.extend(get_fastq_records(os.path.join(filepath, file)))

999it [00:00, 26583.79it/s]0<?, ?it/s]
1691it [00:00, 12118.79it/s]
897it [00:00, 27340.84it/s]0<00:08, 10.81it/s]
806it [00:00, 16918.86it/s]
598it [00:00, 23327.91it/s]
504it [00:00, 22270.41it/s]0<00:04, 17.66it/s]
440it [00:00, 20927.52it/s]
373it [00:00, 18354.85it/s]
344it [00:00, 17497.25it/s]
352it [00:00, 15971.39it/s]0<00:03, 25.73it/s]
303it [00:00, 12038.33it/s]
360it [00:00, 8130.60it/s]
1615it [00:00, 29854.74it/s]0<00:02, 26.78it/s]
635it [00:00, 24777.96it/s]
594it [00:00, 22310.13it/s]
513it [00:00, 21759.84it/s]00<00:02, 26.34it/s]
429it [00:00, 17124.17it/s]
512it [00:00, 22423.34it/s]
469it [00:00, 19127.49it/s]
424it [00:00, 17720.94it/s]00<00:02, 29.43it/s]
429it [00:00, 20405.49it/s]
290it [00:00, 13642.61it/s]
453it [00:00, 12904.59it/s]
1529it [00:00, 31473.13it/s]0<00:02, 31.10it/s]
421it [00:00, 12732.47it/s]
326it [00:00, 11229.27it/s]
352it [00:00, 17580.11it/s]
377it [00:00, 19552.05it/s]01<00:02, 29.71it/s]
318it [00:00, 16907.80it/s]
250it [00:00, 13128.

In [161]:
strand_pool = [str(i.seq) for i in records]
strand_ids = [str(i.id) for i in records]

In [165]:
# Removing adapters and filtering length
front_adapter = 'CATGAAGTGGAGTAGCGGCG'
reverse_adapter = 'GCGACCGATGCTCACTGATC'

rev_front = reverse_complement(reverse_adapter)
rev_back = reverse_complement(front_adapter)

cleaned_strand_pool = []
cleaned_strand_ids = []

for ind, i in enumerate(strand_pool):
    
    if front_adapter in i[:20]:
        if reverse_adapter in i[-21:]:
            cleaned_strand = i[20: -21]
            
            if len(cleaned_strand) > 200 and len(cleaned_strand) < 230:
                cleaned_strand_pool.append(cleaned_strand)
                cleaned_strand_ids.append(strand_ids[ind])
    if rev_front in i[:20]:
        if rev_back in i[-21:]:
            cleaned_strand = reverse_complement(i[20:-21])
            if len(cleaned_strand) > 200 and len(cleaned_strand) < 230:
                cleaned_strand_pool.append(cleaned_strand)
                cleaned_strand_ids.append(strand_ids[ind])

In [169]:
obj = Clustering(strand_pool=cleaned_strand_pool, reference_length=208, strand_pool_ids=strand_ids, front_adapter=front_adapter, reverse_adapter=reverse_adapter, n_reference_strands=800)

In [170]:
obj.run_pipeline(remove_adapters=False)

Filtering strands by length
0.18230659025787965 strands filtered out
Clustering strands
Total strands 6849


100%|██████████| 6849/6849 [00:20<00:00, 328.55it/s]


Number of clusters = 966
Clusters are sorted
Orientation fixed in the strand pool
Generating 1000 candidates


100%|██████████| 850/850 [01:07<00:00, 12.68it/s]


Evaluating candidates


0it [00:00, ?it/s]


IndexError: index 0 is out of bounds for axis 0 with size 0

In [59]:
strand_pool[7]

'CATGAAGTGGAGTAGCGGCGCATGGAGTGGATAACTGCGGTAACTGATCCACCATGTCTCATAGGTCAACGTACTAGAGGGCTTACCTAAGGCGAACCTTAGCATTTATCAGAGTGAGCCGAGCATTGATTAGACCTATGAGACATGGTGGATCAGTTACCGCAGTTATCCCACTCCATGCGCCACTACTCCACTCATGA'

In [ ]:
front_adapter

'CATGAAGTGGAGTAGCGGCG'

In [92]:
df = pd.read_csv(r"C:\Users\Parv\Doc\RA\Projects\sequencingData\JPEGDNA\cat_FFDX_checksum.csv")

In [107]:
len(original_strands)

1031

In [98]:
check = CheckSum4(reference_length=204)

In [171]:
len(set(obj.candidates).intersection(set(original_strands)))

386

In [173]:
decoded_strands = check.decode(obj.candidates, n_reference_strands=1031, clustered_seqs=obj.clustered_seqs, n_guesses=5, guesses=True)

100%|██████████| 850/850 [00:00<00:00, 16185.77it/s]


394 direct checksum matches found
Making guesses


100%|██████████| 456/456 [02:09<00:00,  3.51it/s]

26 extra found after guessing
0.002380952380952381 were reversed
420 Valid checksum strands found


In [133]:
original_strands_ = [i[:-4] for i in original_strands]

In [135]:
len(set(decoded_strands).intersection(set(original_strands_)))

882

In [140]:
len(original_strands)

1031

In [145]:
create_fasta_file(ids=[str(uuid.uuid4()) for i in range(len(decoded_strands))], strands=decoded_strands, output_filepath=r"C:\Users\Parv\Doc\RA\Projects\sequencingData\JPEGDNA\1_Twist_decoded.fasta")

File saved as C:\Users\Parv\Doc\RA\Projects\sequencingData\JPEGDNA\1_Twist_decoded.fasta
